<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=60b206714e29299cb7eb6e92ea9fec34e0f6310385c9583db73e526ff220ae21
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-29 15:11:02
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.28 L
Current:  1.37 C
-------------------
Today PnL: 22.87 K (0.17%)
Current PnL: -20.61 L (-13.86%)
CY Booked + Current PnL: -11.57 L (-7.78%)
-------------------
Total profit:  2.62 L
Total loss:  -23.23 L
-------------------
Total Booked + Current PnL: 15.41 L (12.66%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.62%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 81.99 L (60.01%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.35%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-18.25,56.0,H-LC,7.62,112560.0,8248.0,8397.0,0.30,7.91,7.46,15.96,9.0,0.98,0.85,26.14,X5K,ATH,METALS
53,MEDANTA,1486.00,11.99,56.0,H-MC,15.00,159999.0,33799.0,12384.0,-1.47,26.78,7.74,36.59,108.0,2.73,1.21,42.70,XY24,NTT,HEALTHCARE
81,TTKPRESTIG,770.00,100.00,60.0,M-SC,8.16,86146.0,-14631.0,14722.0,1.61,-14.52,17.09,0.09,245.0,-0.99,0.65,13.18,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-16.19,52.0,H-LC,8.36,167431.0,11254.0,14834.0,-1.52,7.21,8.86,16.71,22.0,0.76,1.27,25.90,X40N,BTT,HEALTHCARE
66,SBIN,863.00,-14.20,41.0,M-LC,4.25,212636.0,11156.0,16054.0,0.06,5.54,7.55,13.51,60.0,0.69,1.61,18.83,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,ICICIPRULI,790.00,-21.22,35.0,H-MC,2.39,135623.0,-165.0,42911.0,-1.55,-0.12,31.64,31.48,107.0,-0.00,1.02,11.98,X40,ATH,INSURANCE
38,ICICIGI,2260.25,-20.81,35.0,H-MC,4.90,136234.0,101.0,31020.0,-1.04,0.07,22.77,22.86,91.0,0.00,1.03,12.70,X40,ATH,INSURANCE
83,UNIONBANK,163.00,-20.93,30.0,M-LC,10.86,141807.0,967.0,43521.0,-1.86,0.69,30.69,31.59,66.0,0.02,1.07,27.36,XY24,NTT,BANKS


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.49,32.0,H-LC,13.49,240168.0,-21451.0,140666.0,0.06,-8.20,58.57,45.57,7.0,-0.15,1.81,2.19,AR,NTT,ELECTRICAL
33,HCLTECH,1943.91,0.22,38.0,H-LC,10.21,228466.0,-13458.0,76719.0,0.37,-5.56,33.58,26.15,8.0,-0.18,1.73,7.96,X40,ATH,IT
47,JIOFIN,387.00,-1.23,40.0,H-LC,15.23,218400.0,994.0,52503.0,0.61,0.46,24.04,24.61,48.0,0.02,1.65,55.53,XY24,BTT,FINANCE
87,VBL,671.64,-13.57,41.0,H-LC,8.07,311121.0,-4721.0,117386.0,-2.02,-1.49,37.73,35.67,5.0,-0.04,2.35,12.15,X40N,ATH,FMCG
43,INFY,2275.00,-19.60,43.0,H-LC,11.76,312940.0,52.0,171648.0,-2.06,0.02,54.85,54.87,3.0,0.00,2.36,6.67,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,NESTLEIND,1377.0,-12.04,51.0,H-LC,2.82,270668.0,5242.0,51562.0,-0.49,1.97,19.05,21.40,11.0,0.10,2.05,9.45,XY25,NTT,FMCG
51,LTIM,7230.2,-6.94,48.0,H-LC,2.93,230602.0,-15549.0,94754.0,0.03,-6.32,41.09,32.18,16.0,-0.16,1.74,27.45,X200,ATH,IT
45,ITC,452.0,-37.92,50.0,H-LC,2.99,200246.0,108.0,20786.0,2.15,0.05,10.38,10.44,4.0,0.01,1.51,5.62,X40,NTT,FMCG
20,CIPLA,1795.0,-18.41,63.0,H-LC,4.42,217830.0,13330.0,28078.0,0.61,6.52,12.89,20.25,10.0,0.47,1.65,15.82,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-12.28,57.0,H-LC,7.23,221487.0,-30281.0,152516.0,1.12,-12.03,68.86,48.55,27.0,-0.20,1.67,19.14,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-35.61,76.0,L-SC,27.26,95424.0,-18125.0,58180.0,16.83,-15.96,60.97,35.27,268.0,-0.31,0.72,133.55,XR,NTT,HOTELS
60,RELAXO,1176.00,-33.78,58.0,H-SC,8.51,61440.0,-54098.0,89088.0,7.21,-46.82,145.00,30.28,136.0,-0.61,0.46,20.79,X40N,NTT,FOOTWEAR
64,SAMMAANCAP,326.00,-164.77,53.0,M-SC,8.23,74694.0,-27516.0,120907.0,5.53,-26.92,161.87,91.37,208.0,-0.23,0.56,21.25,XY25,NTT,FINANCE
23,COLPAL,3767.14,-0.15,57.0,H-MC,10.61,182005.0,-29736.0,111824.0,3.12,-14.04,61.44,38.77,84.0,-0.27,1.38,8.33,XY25,ATH,FMCG
84,UNITDSPR,1644.00,-14.09,46.0,H-LC,8.69,172960.0,-8799.0,44053.0,2.25,-4.84,25.47,19.39,86.0,-0.20,1.31,2.91,X40N,NTT,BREWERIES


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VAIBHAVGBL,521.00,32.23,31.0,H-SC,12.20,88128.0,-63863.0,137982.0,-3.87,-42.02,156.57,48.77,125.0,-0.46,0.67,2.58,XR,NTT,JEWELLERY
43,INFY,2275.00,-19.60,43.0,H-LC,11.76,312940.0,52.0,171648.0,-2.06,0.02,54.85,54.87,3.0,0.00,2.36,6.67,X40,BTT,IT
87,VBL,671.64,-13.57,41.0,H-LC,8.07,311121.0,-4721.0,117386.0,-2.02,-1.49,37.73,35.67,5.0,-0.04,2.35,12.15,X40N,ATH,FMCG
36,HINDZINC,730.22,12.53,43.0,M-LC,4.02,183440.0,-21636.0,134205.0,-1.96,-10.55,73.16,54.89,52.0,-0.16,1.39,11.00,X5K,ATH,METALS
76,TATAELXSI,9161.00,-26.40,18.0,H-MC,6.42,73332.0,-29320.0,54918.0,-1.87,-28.56,74.89,24.94,98.0,-0.53,0.55,11.47,OX40N,NTT,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,100.00,60.0,M-SC,8.16,86146.0,-14631.0,14722.0,1.61,-14.52,17.09,0.09,245.0,-0.99,0.65,13.18,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,83.20,43.0,H-SC,14.62,115938.0,-26541.0,46062.0,1.20,-18.63,39.73,13.70,163.0,-0.58,0.88,36.84,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.70,50.0,H-SC,3.46,215973.0,-53694.0,90018.0,0.81,-19.91,41.68,13.47,138.0,-0.60,1.63,9.24,XY24,NTT,PAINTS
19,CERA,9475.0,-20.53,42.0,H-SC,3.02,113517.0,-30286.0,57031.0,0.95,-21.06,50.24,18.60,149.0,-0.53,0.86,25.27,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,107.59,39.0,M-SC,2.19,136276.0,-38283.0,38321.0,0.15,-21.93,28.12,0.02,221.0,-1.00,1.03,18.54,OX40N,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-27.08,47.0,H-LC,15.60,290281.0,-55675.0,179539.0,-0.18,-16.09,61.85,35.80,1.0,-0.31,2.19,2.83,X40,BTT,IT
43,INFY,2275.00,-19.60,43.0,H-LC,11.76,312940.0,52.0,171648.0,-2.06,0.02,54.85,54.87,3.0,0.00,2.36,6.67,X40,BTT,IT
45,ITC,452.00,-37.92,50.0,H-LC,2.99,200246.0,108.0,20786.0,2.15,0.05,10.38,10.44,4.0,0.01,1.51,5.62,X40,NTT,FMCG
87,VBL,671.64,-13.57,41.0,H-LC,8.07,311121.0,-4721.0,117386.0,-2.02,-1.49,37.73,35.67,5.0,-0.04,2.35,12.15,X40N,ATH,FMCG
1,ABB,7934.00,-42.49,32.0,H-LC,13.49,240168.0,-21451.0,140666.0,0.06,-8.20,58.57,45.57,7.0,-0.15,1.81,2.19,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,11.99,56.0,H-MC,15.00,159999.0,33799.0,12384.0,-1.47,26.78,7.74,36.59,108.0,2.73,1.21,42.7,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-6.70,74.0,H-LC,12.83,233010.0,48529.0,21833.0,1.18,26.31,9.37,38.14,38.0,2.22,1.76,39.7,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,11.99,56.0,H-MC,15.00,159999.0,33799.0,12384.0,-1.47,26.78,7.74,36.59,108.0,2.73,1.21,42.70,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,9.02,59.0,H-MC,4.05,188290.0,23620.0,44342.0,0.15,14.34,23.55,41.28,89.0,0.53,1.42,23.47,X40N,ATH,AC
88,VOLTAS,1918.49,-1.64,59.0,H-MC,6.33,206280.0,14538.0,81501.0,1.17,7.58,39.51,50.08,99.0,0.18,1.56,14.89,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,15.86,178010.0,27097.0,30208.0,-0.11,17.96,16.97,37.97,93.0,0.90,1.34,47.80,AR,ATH,AUTO
24,DABUR,735.00,-6.54,54.0,H-MC,3.78,203307.0,6279.0,83336.0,1.45,3.19,40.99,45.49,102.0,0.08,1.54,16.64,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,15.86,178010.0,27097.0,30208.0,-0.11,17.96,16.97,37.97,93.0,0.90,1.34,47.80,AR,ATH,AUTO
40,INDIAMART,4850.92,-50.72,51.0,H-SC,12.48,137577.0,14241.0,119527.0,0.70,11.55,86.88,108.45,119.0,0.12,1.04,36.48,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-42.31,48.0,M-SC,5.64,97560.0,6062.0,72692.0,0.32,6.62,74.51,86.07,223.0,0.08,0.74,41.67,XR,NTT,DURABLES
90,WIPRO,420.00,-11.98,51.0,M-LC,7.72,154864.0,3919.0,105540.0,-0.35,2.60,68.15,72.51,53.0,0.04,1.17,8.77,XR,NTT,IT
59,RAJOOENG,143.33,-33.56,36.0,H-SC,10.04,100700.0,-1800.0,42626.0,-0.68,-1.76,42.33,39.83,114.0,-0.04,0.76,0.00,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-7.20,35.0,H-SC,16.50,86370.0,-14593.0,109128.0,-0.75,-14.45,126.35,93.63,148.0,-0.13,0.65,28.65,SR,ATH,CHEMICALS
52,MASFIN,398.61,-18.25,51.0,H-SC,12.74,93120.0,-4860.0,26465.0,1.26,-4.96,28.42,22.05,152.0,-0.18,0.70,35.36,XR,ATH,FINANCE
22,COFFEEDAY,80.00,-35.61,76.0,L-SC,27.26,95424.0,-18125.0,58180.0,16.83,-15.96,60.97,35.27,268.0,-0.31,0.72,133.55,XR,NTT,HOTELS
35,HINDALCO,756.01,-18.25,56.0,H-LC,7.62,112560.0,8248.0,8397.0,0.30,7.91,7.46,15.96,9.0,0.98,0.85,26.14,X5K,ATH,METALS
82,UJJIVANSFB,60.00,83.20,43.0,H-SC,14.62,115938.0,-26541.0,46062.0,1.20,-18.63,39.73,13.70,163.0,-0.58,0.88,36.84,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-35.61,76.0,L-SC,27.26,95424.0,-18125.0,58180.0,16.83,-15.96,60.97,35.27,268.0,-0.31,0.72,133.55,XR,NTT,HOTELS
27,DMART,5201.00,-6.70,74.0,H-LC,12.83,233010.0,48529.0,21833.0,1.18,26.31,9.37,38.14,38.0,2.22,1.76,39.70,X40N,NTT,FMCG
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,15.86,178010.0,27097.0,30208.0,-0.11,17.96,16.97,37.97,93.0,0.90,1.34,47.80,AR,ATH,AUTO
14,BLUESTARCO,2326.38,9.02,59.0,H-MC,4.05,188290.0,23620.0,44342.0,0.15,14.34,23.55,41.28,89.0,0.53,1.42,23.47,X40N,ATH,AC
23,COLPAL,3767.14,-0.15,57.0,H-MC,10.61,182005.0,-29736.0,111824.0,3.12,-14.04,61.44,38.77,84.0,-0.27,1.38,8.33,XY25,ATH,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.63
1,25,43.10
2,50,72.66


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.05
LC    36.78
MC    26.14
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.10
X40      15.48
X40N     14.27
XY25      9.80
AR        9.29
XR        9.00
OX40N     7.32
X5K       2.24
X200      1.74
MH        1.66
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.65
H-MC    23.29
H-SC    22.26
M-SC    13.16
M-LC     7.08
M-MC     2.53
L-SC     1.63
L-LC     1.05
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.23,-1.36,34.38
IT,13.33,-16.91,74.90
BANKS,7.31,-18.04,68.16
FINANCE,7.21,-26.14,72.65
PAINTS,5.96,-16.97,43.15
MISC,5.83,-18.96,87.15
ELECTRICAL,5.65,-10.17,70.90
AUTO,4.53,-12.72,61.75
AC,3.71,1.47,31.90


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2913967.0,23
AR,1248372.0,10
XR,1104140.0,13
X40,917369.0,10
X40N,545801.0,11
OX40N,489077.0,10
XY25,446197.0,7
SR,201134.0,2
X5K,142602.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2826817.0,27
M-SC,1502187.0,17
H-LC,1439598.0,17
H-MC,1293983.0,18
M-LC,443034.0,5
M-MC,342039.0,2
L-SC,245654.0,3
L-MC,60450.0,1
L-LC,45321.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,954692.0,6
M-SC,XY24,880344.0,7
H-SC,AR,669846.0,5
H-LC,X40,601208.0,5
H-SC,XR,536794.0,6
H-MC,XY24,497154.0,5
H-LC,AR,385962.0,2
M-MC,XY24,342039.0,2
H-MC,X40,233578.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
